In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from time import time
from tqdm import tqdm
from util import FileStreamer, DynamicMinMax

from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.metrics import mean_squared_error, max_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [2]:
stream = FileStreamer("./data/creditcard.csv", drop_cols=["Class", "Time"], batch_size=500)
dyn_scaler = DynamicMinMax()

In [3]:
partial_pca = IncrementalPCA()

In [4]:
err= []
for batch in tqdm(stream):
    dyn_scaler.fit(batch)
    batch = dyn_scaler.transform(batch)
    
    partial_pca.partial_fit(batch)
    tbatch = partial_pca.transform(batch)
    rev = partial_pca.inverse_transform(tbatch)
    
    err.append(mean_squared_error(batch, rev))
np.mean(err)

570it [00:05, 95.90it/s]                                                                                                                                                                                        


8.669475590101916e-33

In [5]:
np.max(err)

4.153669840976501e-32

In [6]:
scaler = MinMaxScaler().fit(stream.data)
pca = PCA().fit( scaler.transform(stream.data) )

tX = pca.transform(scaler.transform(stream.data) )
rev = pca.inverse_transform(tX)

mean_squared_error(rev, scaler.transform(stream.data))

2.2483096465938877e-33

In [7]:
pca.transform(scaler.transform(stream.data)).mean()

5.193961073399137e-20

In [8]:
partial_pca.transform(dyn_scaler.transform(stream.data)).mean()

-0.0003176869269594927

- Reconstruction error with the same numerical order of the offline PCA. 

Compare the procedure when the data is not scaled

In [10]:
partial_pca = IncrementalPCA()

In [11]:
err= []
for batch in tqdm(stream):
    partial_pca.partial_fit(batch)
    tbatch = partial_pca.transform(batch)
    rev = partial_pca.inverse_transform(tbatch)
    
    err.append(mean_squared_error(batch, rev))
np.mean(err)

570it [00:04, 116.75it/s]                                                                                                                                                                                       


9.64945385228029e-28

In [12]:
np.max(err)

3.446545423530526e-26

In [13]:
pca = PCA().fit( stream.data )
tX = pca.transform( stream.data )
rev = pca.inverse_transform(tX)

mean_squared_error(rev, stream.data)

5.287907178069302e-28

In [14]:
pca.transform(stream.data).mean()

2.767772601610861e-16

In [15]:
partial_pca.transform(stream.data).mean()

1.2251536887019074e-15

Similar Results.

Overall, Incremental PCA seems like a good aproximation to the default PCA